In [10]:
import pandas as pd
import numpy as np

In [2]:
df=pd.read_csv('fulldata.csv')

In [3]:
df

,Fat,Fiber,Carbohydrates,Energy,Proteins,Salts,Sugars,Positive Points,Negative Points,Blood Pressure Suitability,Diabetes Suitability,Cholestrol Suitability,Skinny,Fit,Obese
0,1.80,11.1,77.4,1.0,10.80,0.000,5.6,10.0,5.0,0,1,1,1,0,0
1,25.00,0.0,62.0,2.0,7.40,0.686,42.3,0.0,26.0,0,0,0,0,0,0
2,3.00,0.0,4.8,244.0,3.00,0.092,4.8,1.0,2.0,1,1,1,1,1,1
3,4.10,0.0,13.4,452.0,3.60,0.170,12.9,2.0,5.0,0,0,1,0,0,0
4,0.56,0.0,66.6,1.0,1.27,6.000,63.7,0.0,23.0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199,1.50,0.0,5.0,192.0,3.20,0.105,5.0,1.0,1.0,1,1,1,1,1,1
200,0.00,0.0,0.0,0.0,0.00,0.003,0.0,0.0,0.0,1,1,1,1,1,1
201,28.90,0.0,60.4,2.0,8.10,0.156,57.3,0.0,26.0,0,0,0,0,0,0
202,34.60,0.0,55.2,2.0,6.40,2.230,1.0,0.0,25.0,0,1,0,0,0,0


In [6]:
df['safety_score']=df['Positive Points']-df['Negative Points']

In [13]:
import pandas as pd

def scale_to_range(value, min_value, max_value, target_min=-5, target_max=5):
    proportion = (value - min_value) / (max_value - min_value)
    scaled_value = proportion * (target_max - target_min) + target_min
    return scaled_value


data = df['safety_score']
min_value = data.min()
max_value = data.max()

scaled_data = data.apply(lambda x: scale_to_range(x, min_value, max_value))

print("Scaled Data (Range -5 to +5):")
print(scaled_data)


Scaled Data (Range -5 to +5):
0      4.459459
1     -3.918919
2      2.837838
3      2.297297
4     -3.108108
         ...   
199    3.108108
200    3.108108
201   -3.918919
202   -3.648649
203   -3.918919
Name: safety_score, Length: 204, dtype: float64


In [19]:
df['scaled_data']=scaled_data

In [21]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error


In [22]:
X=df[['Fat', 'Fiber', 'Carbohydrates', 'Energy', 'Proteins', 'Salts','Sugars']]
y=df[['scaled_data']]

In [23]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [25]:
rf_regressor = RandomForestRegressor(n_estimators=100, random_state=42)

In [26]:
rf_regressor.fit(X_train, y_train)

c:\Users\sriyo\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:1152: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


RandomForestRegressor(random_state=42)

In [27]:
y_pred = rf_regressor.predict(X_test)


In [28]:
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)


Mean Squared Error: 1.2694295750265765


In [29]:
from sklearn.metrics import r2_score

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
accuracy = r2 * 100
print("Accuracy (R^2 score): {:.2f}%".format(accuracy))


Accuracy (R^2 score): 78.31%


In [52]:
custom_input = [[19,2.7,68,1990,5.4,0.74,38]] 
predictions =rf_regressor.predict(custom_input)
print("Predictions:", predictions)

Predictions: [-2.73783784]


c:\Users\sriyo\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


In [51]:
custom_input = [[7.2,9.6,68,407,12.8,9.8,0]] 
predictions =rf_regressor.predict(custom_input)
print("Predictions:", predictions)

Predictions: [0.62432432]


c:\Users\sriyo\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


In [50]:
custom_input = [[18,0,73.4,479,5.9,0,32.5]] 
predictions =rf_regressor.predict(custom_input)
print("Predictions:", predictions)

Predictions: [-2.12162162]


c:\Users\sriyo\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


In [55]:
import joblib
joblib.dump(rf_regressor, 'safetyscore.pkl')


['safetyscore.pkl']

## XGBOOST

In [30]:
import xgboost as xgb

In [36]:
model = xgb.XGBRegressor(
    n_estimators=100,  # Number of boosting rounds
    max_depth=3,       # Maximum tree depth
    learning_rate=0.1, # Learning rate
    random_state=42    # Random seed for reproducibility
)
model.fit(X_train, y_train)


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=3, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=100, n_jobs=None,
             num_parallel_tree=None, random_state=42, ...)

In [37]:
y_pred = model.predict(X_test)


In [38]:
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)

In [41]:
from sklearn.metrics import r2_score

# Calculate the R^2 score
r2 = r2_score(y_test, y_pred)

# Convert R^2 score to percentage
accuracy_score = r2 * 100

print("Accuracy Score: {:.2f}".format(accuracy_score))


Accuracy Score: 75.47


In [43]:
from sklearn.model_selection import GridSearchCV

params = {
    'max_depth': [3, 6, 9],
    'learning_rate': [0.1, 0.01, 0.001]
}
grid_search = GridSearchCV(model, params, cv=5)
grid_search.fit(X_train, y_train)
best_params = grid_search.best_params_


In [44]:
from xgboost import XGBRegressor

# Instantiate XGBRegressor with the best parameters
best_model = XGBRegressor(**grid_search.best_params_)

# Fit the model to the training data
best_model.fit(X_train, y_train)

# Now you can use the trained model to make predictions
predictions = best_model.predict(X_test)


In [45]:
from sklearn.metrics import r2_score

# Calculate the R^2 score
r2 = r2_score(y_test, predictions)

# Convert R^2 score to percentage
accuracy_score = r2 * 100

print("Accuracy Score: {:.2f}".format(accuracy_score))


Accuracy Score: 75.47
